In [1]:
### @credit: gensim tutorial on LDA and ensembleLda
import gensim
from spacy.lang.en.stop_words import STOP_WORDS
def tokenize(sentence):
    return [token for token in sentence.split() if token not in STOP_WORDS]
import re
def clean_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    return re.sub(r'\s{2,}', ' ', sentence)
from gensim.models.phrases import Phrases, Phraser
def build_phrases(sentences):
    phrases = Phrases(sentences,
                      min_count=1,
                      threshold=10,
                      progress_per=1000)
    return Phraser(phrases)
def sentence_to_bi_grams(phrases_model, sentence):
    return ' '.join(phrases_model[sentence])


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
for handler in logger.handlers:
    handler.setLevel(logging.DEBUG)

In [3]:
p = []
with open("phrases.txt") as f:
    for line in f:
        p.append(line.split())
p = dict(p)

In [19]:
docs = []
# with open("textretrieval.txt", encoding='utf-8') as f:
#     for line in f:
#         docs.append(line.split('.'))
# with open("textanalytics.txt") as f:
#     for line in f:
#         docs.append(line.split('.'))
with open("noncs.dat", encoding='utf-8') as f:
    for line in f:
        docs.append(line.split('.'))
with open("cs125.dat", encoding='utf-8') as f:
    for line in f:
        docs.append(line.split('.'))
with open("bkgd.dat", encoding='utf-8') as f:
    for line in f:
        docs.append(line.split('.'))
print(len(docs))
docs = [[clean_sentence(s) for s in sentences] for sentences in docs]
docs = [[tokenize(s) for s in sentences] for sentences in docs]

phrased = []
for sentences in docs:
    ps = []
    for s in sentences:
        i = 0
        new_s = []
        while i < len(s) - 1:
            if (s[i+1] == p.get(s[i], False)):
                new_s.append(s[i] + "_" + s[i+1])
                i+=2
            else:
                new_s.append(s[i])
                i+=1
        ps.append(new_s)
    phrased.append(ps)
len(phrased)

64


64

In [20]:
phrased_doc = []
for doc in phrased:
    d = []
    for s in doc:
        d+=s
    phrased_doc.append(d)
phrased_doc

[['history',
  'carry',
  'words',
  'youre',
  'product',
  'term',
  'usually',
  'relegated',
  'people',
  'lowincome',
  'crimeinfested',
  'areasbut',
  'product',
  'existence',
  'amalgamation',
  'triumph',
  'hardwon',
  'battle',
  'woman',
  'idea',
  'massaged',
  'courage',
  'use',
  'change',
  'person',
  'survived',
  'slavery',
  'jim',
  'crow',
  'black',
  'codes',
  'trail',
  'tears',
  'warsand',
  'passed',
  'dreams',
  'youof',
  'love',
  'yup',
  'product',
  'silence',
  'apathy',
  'school',
  'built',
  'stolen',
  'women',
  'parent',
  'grandparent',
  'ancestor',
  'suppressed',
  'dreams',
  'ideas',
  'died',
  'lost',
  'potential',
  'horrific',
  'memories',
  'sexual',
  'assault',
  'mental',
  'illness',
  'didnt',
  'feel',
  'good',
  'anxiety',
  'privileged',
  'blessedto',
  'dowhat',
  'roads',
  'people',
  'usually',
  'choice',
  'think',
  'path',
  'success',
  'high',
  'climb',
  'career',
  'socalled',
  'save',
  'world',
  'ap

In [21]:
with open("miss.dat", "w") as f:
    for doc in phrased_doc:
        f.write(" ".join(doc)+"\n")

In [9]:
from gensim.models import LdaModel

In [10]:
from gensim.corpora import Dictionary
import random

dictionary = Dictionary(phrased_doc)
dictionary.filter_extremes(no_below=2, no_above=0.1)

corpus = [dictionary.doc2bow(doc) for doc in phrased_doc]
num_topics = 86*2
chunksize = 160 # how many documents to process at a time
passes = 20 # epochs
iterations = 30
eval_every = 10

# Make a index to word dictionary.
temp = dictionary[0] 
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)


2021-11-27 22:19:50,821 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2021-11-27 22:19:50,912 : INFO : built Dictionary<12250 unique tokens: ['1st', '90', '97', 'able', 'accuracy']...> from 160 documents (total 216470 corpus positions)
2021-11-27 22:19:50,912 : DEBUG : starting a new internal lifecycle event log for Dictionary
2021-11-27 22:19:50,913 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12250 unique tokens: ['1st', '90', '97', 'able', 'accuracy']...> from 160 documents (total 216470 corpus positions)", 'datetime': '2021-11-27T22:19:50.912950', 'gensim': '4.1.3.dev0', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2021-11-27 22:19:50,922 : INFO : discarding 6758 tokens: [('able', 87), ('accuracy', 17), ('achieve', 35), ('acquired', 1), ('act', 18), ('actually', 136), ('add', 61), ('additional', 36), ('allow', 73), ('alright', 46)]...
2021-11-27 22:19:50

2021-11-27 22:19:51,718 : DEBUG : bound: at document #0
2021-11-27 22:19:51,941 : INFO : -18.731 per-word bound, 434977.8 perplexity estimate based on a held-out corpus of 160 documents with 51688 words
2021-11-27 22:19:51,941 : INFO : PROGRESS: pass 1, at document #160/160
2021-11-27 22:19:51,942 : DEBUG : performing inference on a chunk of 160 documents
2021-11-27 22:19:52,098 : DEBUG : 113/160 documents converged within 30 iterations
2021-11-27 22:19:52,101 : INFO : optimized alpha [0.0067588715, 0.00639538, 0.0065924684, 0.0067378487, 0.006165688, 0.0064504817, 0.0060319714, 0.0063254926, 0.0065959846, 0.006596367, 0.0062809675, 0.0064463047, 0.006198315, 0.007355833, 0.007243345, 0.0062428606, 0.0063047986, 0.0064690583, 0.0062282667, 0.0062837275, 0.0064209886, 0.0067944406, 0.006621338, 0.006263394, 0.0062370626, 0.0067517175, 0.0063578836, 0.0062849205, 0.006876836, 0.006026779, 0.005921303, 0.0066210055, 0.0067302063, 0.0064238897, 0.006568685, 0.006113791, 0.0066861417, 0.007

2021-11-27 22:19:52,906 : INFO : -10.420 per-word bound, 1369.9 perplexity estimate based on a held-out corpus of 160 documents with 51688 words
2021-11-27 22:19:52,906 : INFO : PROGRESS: pass 3, at document #160/160
2021-11-27 22:19:52,907 : DEBUG : performing inference on a chunk of 160 documents
2021-11-27 22:19:53,003 : DEBUG : 154/160 documents converged within 30 iterations
2021-11-27 22:19:53,006 : INFO : optimized alpha [0.0071423003, 0.006175263, 0.006667901, 0.00667366, 0.00599768, 0.0064153965, 0.0059595755, 0.0061101187, 0.006575533, 0.006362327, 0.0060685896, 0.006297899, 0.0059914435, 0.008878054, 0.008087867, 0.0061063757, 0.0060908194, 0.0063780556, 0.006484346, 0.006071165, 0.006275397, 0.006992309, 0.0067970813, 0.0065569277, 0.0060276184, 0.0075696125, 0.006291653, 0.0060722777, 0.007713719, 0.0061187926, 0.005732363, 0.00658138, 0.0073013455, 0.0063153096, 0.006768213, 0.0059124753, 0.0068238685, 0.008470517, 0.006404451, 0.006186452, 0.006424585, 0.005924819, 0.005

2021-11-27 22:19:53,818 : INFO : PROGRESS: pass 5, at document #160/160
2021-11-27 22:19:53,818 : DEBUG : performing inference on a chunk of 160 documents
2021-11-27 22:19:53,906 : DEBUG : 159/160 documents converged within 30 iterations
2021-11-27 22:19:53,909 : INFO : optimized alpha [0.0074295485, 0.0060046404, 0.006764292, 0.00658944, 0.005866298, 0.0064207376, 0.0059460513, 0.0059430515, 0.006576069, 0.006181288, 0.0059037707, 0.0061816694, 0.0058307596, 0.010307026, 0.008815318, 0.005999109, 0.005924799, 0.0063228807, 0.0066820392, 0.0059062075, 0.0061614173, 0.0071538324, 0.0069093, 0.006853259, 0.0058650034, 0.008298493, 0.006239599, 0.0059072613, 0.00838161, 0.0061940444, 0.005585191, 0.006549869, 0.007775123, 0.0062299063, 0.006932847, 0.005755974, 0.0069534914, 0.0094659915, 0.0063480153, 0.006166684, 0.006398398, 0.005767666, 0.0057869977, 0.006352764, 0.006232124, 0.0060075796, 0.006255103, 0.006866398, 0.0058359774, 0.0072439965, 0.0075078695, 0.006032441, 0.0064312443, 0

2021-11-27 22:19:54,800 : DEBUG : 157/160 documents converged within 30 iterations
2021-11-27 22:19:54,803 : INFO : optimized alpha [0.007665653, 0.0058636777, 0.0068329168, 0.0065046824, 0.0057570357, 0.0064391694, 0.0059612165, 0.00580495, 0.0065628886, 0.0060319635, 0.0057674767, 0.0060849464, 0.005697797, 0.011647924, 0.009443084, 0.005909667, 0.0057875384, 0.006303746, 0.006855305, 0.0057698027, 0.0060665347, 0.007294082, 0.0069909296, 0.0071162633, 0.0057304846, 0.008916337, 0.0062225997, 0.005770809, 0.008975395, 0.00628564, 0.005463132, 0.0065234657, 0.008253095, 0.006158584, 0.007076061, 0.005626382, 0.007065936, 0.010314203, 0.0063007753, 0.006150196, 0.0063765417, 0.00563755, 0.005656012, 0.006304738, 0.006213865, 0.005866483, 0.0061559146, 0.007037944, 0.005702779, 0.0075136423, 0.007778351, 0.0059679784, 0.0065765176, 0.008136657, 0.0060337996, 0.006484265, 0.005739909, 0.00634132, 0.007315787, 0.0061815814, 0.005896807, 0.006507938, 0.007600145, 0.0069789356, 0.0074478397

...

2021-11-27 22:19:56,578 : DEBUG : updating topics
2021-11-27 22:19:56,680 : INFO : topic #30 (0.005): 0.000*"grade" + 0.000*"compiler" + 0.000*"condition" + 0.000*"z" + 0.000*"bar" + 0.000*"sampling" + 0.000*"opinion" + 0.000*"quizzes" + 0.000*"app" + 0.000*"deviation"
2021-11-27 22:19:56,680 : INFO : topic #98 (0.005): 0.000*"z" + 0.000*"dog" + 0.000*"pet" + 0.000*"dad" + 0.000*"bar" + 0.000*"sampling" + 0.000*"population" + 0.000*"statistics" + 0.000*"taught" + 0.000*"deviation"
2021-11-27 22:19:56,681 : INFO : topic #14 (0.011): 0.066*"clustering" + 0.065*"cluster" + 0.032*"training_data" + 0.027*"theta" + 0.018*"clusters" + 0.016*"generative" + 0.013*"categorization" + 0.012*"errors" + 0.011*"labels" + 0.010*"label"
2021-11-27 22:19:56,682 : INFO : topic #37 (0.012): 0.075*"sorted" + 0.069*"merge" + 0.028*"sorting" + 0.026*"recursive" + 0.018*"smallest" + 0.017*"insertion" + 0.013*"o" + 0.010*"copy" + 0.010*"base" + 0.010*"merges"
2021-11-27 22:19:56,682 : INFO : topic #13 (0.014):

2021-11-27 22:19:57,595 : INFO : topic #98 (0.005): 0.000*"z" + 0.000*"dog" + 0.000*"pet" + 0.000*"dad" + 0.000*"bar" + 0.000*"sampling" + 0.000*"population" + 0.000*"statistics" + 0.000*"taught" + 0.000*"deviation"
2021-11-27 22:19:57,596 : INFO : topic #14 (0.011): 0.066*"clustering" + 0.063*"cluster" + 0.032*"training_data" + 0.026*"theta" + 0.018*"clusters" + 0.016*"generative" + 0.012*"categorization" + 0.012*"errors" + 0.011*"labels" + 0.011*"classify"
2021-11-27 22:19:57,596 : INFO : topic #37 (0.013): 0.076*"sorted" + 0.070*"merge" + 0.027*"sorting" + 0.026*"recursive" + 0.018*"smallest" + 0.017*"insertion" + 0.013*"o" + 0.010*"base" + 0.010*"copy" + 0.010*"merges"
2021-11-27 22:19:57,597 : INFO : topic #13 (0.015): 0.066*"interface" + 0.030*"comparable" + 0.023*"interfaces" + 0.018*"implements" + 0.011*"lowercase" + 0.011*"broken" + 0.009*"push" + 0.009*"dictionary" + 0.008*"implementing" + 0.008*"ad"
2021-11-27 22:19:57,600 : INFO : topic diff=0.912962, rho=0.258199
2021-11-2

2021-11-27 22:19:58,540 : INFO : topic #14 (0.012): 0.066*"clustering" + 0.063*"cluster" + 0.033*"training_data" + 0.026*"theta" + 0.018*"clusters" + 0.016*"generative" + 0.012*"errors" + 0.012*"categorization" + 0.011*"labels" + 0.011*"classify"
2021-11-27 22:19:58,541 : INFO : topic #37 (0.013): 0.076*"sorted" + 0.070*"merge" + 0.027*"recursive" + 0.027*"sorting" + 0.018*"smallest" + 0.017*"insertion" + 0.013*"o" + 0.011*"base" + 0.011*"copy" + 0.010*"merges"
2021-11-27 22:19:58,542 : INFO : topic #13 (0.017): 0.064*"interface" + 0.029*"comparable" + 0.022*"interfaces" + 0.017*"implements" + 0.011*"lowercase" + 0.011*"broken" + 0.009*"push" + 0.009*"dictionary" + 0.008*"implementing" + 0.008*"ad"
2021-11-27 22:19:58,544 : INFO : topic diff=0.544637, rho=0.242536
2021-11-27 22:19:58,604 : DEBUG : bound: at document #0
2021-11-27 22:19:58,783 : INFO : -8.261 per-word bound, 306.8 perplexity estimate based on a held-out corpus of 160 documents with 51688 words
2021-11-27 22:19:58,783 : 

2021-11-27 22:19:59,429 : INFO : topic #37 (0.014): 0.076*"sorted" + 0.070*"merge" + 0.027*"recursive" + 0.026*"sorting" + 0.018*"smallest" + 0.017*"insertion" + 0.012*"o" + 0.011*"base" + 0.011*"copy" + 0.010*"merges"
2021-11-27 22:19:59,429 : INFO : topic #13 (0.018): 0.062*"interface" + 0.029*"comparable" + 0.020*"interfaces" + 0.017*"implements" + 0.012*"lowercase" + 0.011*"broken" + 0.009*"push" + 0.009*"dictionary" + 0.008*"implementing" + 0.008*"unit"
2021-11-27 22:19:59,432 : INFO : topic diff=0.332080, rho=0.229416
2021-11-27 22:19:59,490 : DEBUG : bound: at document #0
2021-11-27 22:19:59,669 : INFO : -8.249 per-word bound, 304.2 perplexity estimate based on a held-out corpus of 160 documents with 51688 words
2021-11-27 22:19:59,669 : INFO : PROGRESS: pass 18, at document #160/160
2021-11-27 22:19:59,670 : DEBUG : performing inference on a chunk of 160 documents
2021-11-27 22:19:59,758 : DEBUG : 160/160 documents converged within 30 iterations
2021-11-27 22:19:59,762 : INFO :

2021-11-27 22:20:00,315 : INFO : topic #13 (0.019): 0.059*"interface" + 0.028*"comparable" + 0.019*"interfaces" + 0.016*"implements" + 0.012*"lowercase" + 0.011*"broken" + 0.009*"dictionary" + 0.009*"push" + 0.008*"unit" + 0.008*"implementing"
2021-11-27 22:20:00,317 : INFO : topic diff=0.207428, rho=0.218218
2021-11-27 22:20:00,348 : DEBUG : starting a new internal lifecycle event log for LdaModel
2021-11-27 22:20:00,348 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel<num_terms=5492, num_topics=172, decay=0.5, chunksize=160> in 9.24s', 'datetime': '2021-11-27T22:20:00.348451', 'gensim': '4.1.3.dev0', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [14]:
print(len(phrased_doc))
i = 12
i-=1
top_topics = model.get_document_topics(corpus[i]) # [(topic_id, prob)]
print(phrased_doc[i])
####
top_topics.sort(key=takelast, reverse=True)
for topic in top_topics:
    print('topic {}'.format(i))
    i+=1
    topic_term_distribution = model.get_topic_terms(topic[0])
    for term in topic_term_distribution:
        print('{}, {}'.format(dictionary[term[0]], term[1]))
    print('prob: {}'.format(topic[-1]))

160
['lecture', 'fast_search', 'inverted_index', 'lecture', 'going', 'continue', 'discussion', 'system', 'particular', 'going', 'talk', 'support', 'fast_search', 'inverted_index', 'lets', 'think', 'general', 'scoring_function', 'look', 'course', 'vector_space', 'model', 'special', 'case', 'imagine', 'retrieval_functions', 'form', 'function', 'scoring_function', 'document', 'd', 'query', 'q', 'defined', 'function', 'thats', 'adjustment', 'function', 'consider', 'factors', 'shown', 'end', 'f', 'sub', 'd', 'd', 'f', 'sub', 'q', 'adjustment', 'factors', 'document', 'query', 'level', 'document', 'inside', 'function', 'theres', 'function', 'called', 'main', 'scoring_function', 'said', 'scoring', 'factors', 'level', 'document', 'example', 'document', 'aggregate', 'functioning', 'inside', 'edge', 'function', 'functions', 'compute', 'weights', 'contribution', 'matched', 'query', 'term', 'function', 'g', 'gives', 'weight', 'match', 'query', 'term', 'ti', 'document', 'h', 'function', 'aggregate',

In [12]:
def takelast(x):
    return x[-1]

In [13]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model2.lda")
model.save(temp_file)

2021-11-27 22:20:07,084 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2021-11-27 22:20:07,084 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2021-11-27 22:20:07,085 : DEBUG : starting a new internal lifecycle event log for Dictionary
2021-11-27 22:20:07,085 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2021-11-27T22:20:07.085097', 'gensim': '4.1.3.dev0', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2021-11-27 22:20:07,137 : DEBUG : starting a new internal lifecycle event log for LdaState
2021-11-27 22:20:07,137 : INFO : LdaState lifecycle event {'fname_or_handle': 'D:\\ProgramFiles\\conda\\lib\\site-packages\\gensim-4.1.3